# 課程目標:
    
    了解Keras 函數式 API 的使用
    

# 範例重點: 
    了解函數式 API 的彈性

In [1]:
import keras
from keras.layers import Input, Embedding, LSTM, Dense
from keras.models import Model

Using TensorFlow backend.


In [3]:
# 主要輸入接收新聞標題本身，即一個整數序列(每個整數編碼一個詞)。
# 這些整數在 1 到 10,000 之間(10,000 個詞的詞彙表)，且序列長度為 100 個詞，並且要宣告一個 NAME 去定義 Input
main_input = Input(shape=(100,), dtype='int32', name='main_input')

# Embedding 層將輸入序列編碼為一個稠密向量的序列，每個向量維度為 512。
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)

# LSTM 層把向量序列轉換成單個向量，它包含整個序列的上下文信息
lstm_out = LSTM(32)(x)

# 插入輔助損失，使得即使在模型主損失很高的情況下，LSTM 層和 Embedding 層都能被平穩地訓練
news_output = Dense(1, activation='sigmoid', name='news_output')(lstm_out)

# 輔助輸入數據與 LSTM 層的輸出連接起來，輸入到模型
news_input = Input(shape=(5,), name='news_input')
merge = keras.layers.concatenate([lstm_out, news_input])

# 堆疊多個全連接網路層
x = Dense(64, activation='relu')(merge)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

# 宣告 MODEL API，分別採用自行定義的 Input/Output Layer
model = Model(inputs=[main_input, news_input], outputs=[main_output, news_output])

# Compile
model.compile(optimizer='rmsprop',
              loss={'main_output': 'binary_crossentropy', 'news_output': 'binary_crossentropy'},
              loss_weights={'main_output': 1., 'news_output': 0.2})

print(model.summary())


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 100)          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 100, 512)     5120000     main_input[0][0]                 
__________________________________________________________________________________________________
lstm_2 (LSTM)                   (None, 32)           69760       embedding_2[0][0]                
__________________________________________________________________________________________________
news_input (InputLayer)         (None, 5)            0                                          

# 作業:
    請修改 Name 中, 自定義的 Layer 名稱
    增加一層全連階層
    宣告 MODEL API, 分別採用自行定義的 Input/Output Layer
    model.summary 查看 Layers stack